In [1]:


#  colab用マウントコードの用意
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#いろいろとインストール
import requests #クローリングのためのモジュール
from bs4 import BeautifulSoup as bs4#HTMLから特定の情報を抜き出すためのモジュール
import time     #サーバーに負荷をかけないためのモジュール
import os       #ディレクトリ作成のためのモジュール
import pandas as pd
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
import lightgbm as lgb
import numpy as np

from sklearn.decomposition import PCA

## 決定版

In [3]:
date='20210101'#日付を入力

race_df=pd.DataFrame(index=[], columns=[])
for i in range(12):
    rno=i+1
    #まず初めに１ページの情報を抜き出す機能
    url='http://www.boatrace.jp/owpc/pc/race/racelist?rno={}&jcd=21&hd={}'.format(rno,date)
    response=requests.get(url)#対象のURLをget
    response.encoding = response.apparent_encoding
    start_page=bs4(response.text, 'html.parser')
    racers_div=start_page.select_one(".is-tableFixed__3rdadd")
    racers_sep_row=racers_div.find_all('tbody') 
    index=0
    race_racers_data=[0]*6
    if len(racers_sep_row)==6:
        for racer_html in racers_sep_row:
            racer_row_ex_td=racer_html.find_all('td')
            #選手の登録ID
            racer_ID_div=racer_row_ex_td[2].find_all('div')
            racer_ID_div=racer_ID_div[0]
            racer_ID_div_txt=racer_ID_div.text
            start=(racer_ID_div_txt.find('/'))-35
            end=(racer_ID_div_txt.find('/')-31)
            racer_ID=racer_ID_div_txt[start:end]#選手の登録ID

            #選手のモータ番号
            racer_moter_td=racer_row_ex_td[6]
            racer_moter_td_text=racer_moter_td.text
            racer_moter_split=racer_moter_td_text.split('\r')
            racer_moter_id=racer_moter_split[1][-2:]
            #選手のボート番号
            racer_boat_td=racer_row_ex_td[7]
            racer_boat_td_text=racer_boat_td.text
            racer_boat_split=racer_boat_td_text.split('\r')
            racer_boat_id=racer_boat_split[1][-2:]
            
            racer_data=[racer_ID,racer_moter_id,racer_boat_id]#選手固有のデータを持ったリスト

            race_racers_data[index]=racer_data
            index+=1
    else:
        print('欠場選手等の例外が発生していないか確認してください')
    #race_racers_dataを一行に変換する
    race_row=pd.Series(index=[])#一レースの情報を一行に変換したもの
    race_num=rno#ここにはレース番号を入れる変数の値を代入
    race_row['number_race']=race_num
    for i in range(len(race_racers_data)):
        race_row['racer_{}_ID'.format(i+1)]=int(race_racers_data[i][0])
        race_row['racer_{}_mo'.format(i+1)]=int(race_racers_data[i][1])
        race_row['racer_{}_bo'.format(i+1)]=int(race_racers_data[i][2])
    race_df=race_df.append(race_row,ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [4]:
url

'http://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=21&hd=20210101'

In [5]:
race_df

,number_race,racer_1_ID,racer_1_bo,racer_1_mo,racer_2_ID,racer_2_bo,racer_2_mo,racer_3_ID,racer_3_bo,racer_3_mo,racer_4_ID,racer_4_bo,racer_4_mo,racer_5_ID,racer_5_bo,racer_5_mo,racer_6_ID,racer_6_bo,racer_6_mo
0,1.0,4477.0,62.0,14.0,4355.0,69.0,27.0,4674.0,75.0,58.0,5029.0,46.0,40.0,3742.0,67.0,16.0,5071.0,59.0,51.0
1,2.0,4193.0,35.0,65.0,4892.0,29.0,2.0,4588.0,60.0,19.0,4831.0,12.0,29.0,4395.0,38.0,4.0,5107.0,48.0,20.0
2,3.0,4474.0,65.0,56.0,4329.0,11.0,1.0,4148.0,20.0,49.0,4555.0,33.0,23.0,4932.0,54.0,9.0,4100.0,55.0,11.0
3,4.0,4522.0,72.0,43.0,3566.0,24.0,17.0,4368.0,58.0,13.0,3747.0,61.0,60.0,5081.0,68.0,42.0,4632.0,56.0,39.0
4,5.0,4545.0,52.0,25.0,4697.0,50.0,45.0,3297.0,49.0,53.0,5071.0,59.0,51.0,3662.0,25.0,6.0,4463.0,66.0,38.0
5,6.0,3444.0,23.0,30.0,4736.0,18.0,18.0,4232.0,53.0,55.0,3742.0,67.0,16.0,5107.0,48.0,20.0,4189.0,47.0,63.0
6,7.0,3434.0,63.0,41.0,4588.0,60.0,19.0,4709.0,16.0,24.0,5041.0,21.0,36.0,4555.0,33.0,23.0,5022.0,17.0,35.0
7,8.0,4402.0,40.0,61.0,4727.0,30.0,5.0,4346.0,57.0,64.0,3783.0,71.0,54.0,4474.0,65.0,56.0,4180.0,36.0,12.0
8,9.0,4395.0,38.0,4.0,4477.0,62.0,14.0,4506.0,37.0,26.0,5081.0,68.0,42.0,5073.0,31.0,21.0,3297.0,49.0,53.0
9,10.0,4424.0,28.0,62.0,3747.0,61.0,60.0,4892.0,29.0,2.0,4189.0,47.0,63.0,4100.0,55.0,11.0,5041.0,21.0,36.0


### 作ったｄｆと選手のパラメータをくっつける

In [6]:
#使用するファイルの定義
#para_file='20'
para_file='21'
para_file_path="/content/drive/My Drive/pred_tool/asiya/racer_pala_box/{}.csv".format(para_file)
#/content/drive/My Drive/boatrace_BOT_making/pred_tool/racer_pala_box/20.csv
#result_file_path="asiya_result_csv/asiya_result_20{0}.csv".format(year)
write_path="/content/drive/My Drive/pred_tool/asiya/pred_data/{0}_asiya.csv".format(date)
#/////////////////////////////////////////////以下データフレームの作成
para_df=pd.read_csv(para_file_path)
para_df=para_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
#出力用データフレーム
#pred_race_df=pd.DataFrame(columns=['result_com','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
pred_race_df=pd.DataFrame(columns=[],index=[])
for index,series in race_df.iterrows():
    add_df=pd.DataFrame(columns=['number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
    #///////////////////////////////////////レースに出ているレーサーの成績を検索＆取得
    ID_1=series['racer_1_ID']
    ID_2=series['racer_2_ID']
    ID_3=series['racer_3_ID']
    ID_4=series['racer_4_ID']
    ID_5=series['racer_5_ID']
    ID_6=series['racer_6_ID']
    racer_1_df=para_df[para_df['racer_ID']==ID_1]
    racer_2_df=para_df[para_df['racer_ID']==ID_2]
    racer_3_df=para_df[para_df['racer_ID']==ID_3]
    racer_4_df=para_df[para_df['racer_ID']==ID_4]
    racer_5_df=para_df[para_df['racer_ID']==ID_5]
    racer_6_df=para_df[para_df['racer_ID']==ID_6]
    if len(racer_1_df)==1:
      pass
    else:
      racer_1_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')

    if len(racer_2_df)==1:
      pass
    else:
      racer_2_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')

    if len(racer_3_df)==1:
      pass
    else:
      racer_3_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')

    if len(racer_4_df)==1:
      pass
    else:
      racer_4_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')
    
    if len(racer_5_df)==1:
      pass
    else:
      racer_5_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')


    if len(racer_6_df)==1:
      pass
    else:
      racer_6_df=para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)]
      print('CAREFULL!!!!    NOT FOUND RACER ')

    #追加していくデータフレームを作成
   
    add_df= pd.DataFrame({'number_race':series['number_race'],
                            'racer_1_ID':series['racer_1_ID'],
                            'racer_2_ID':series['racer_2_ID'],
                            'racer_3_ID':series['racer_3_ID'],
                            'racer_4_ID':series['racer_4_ID'],
                            'racer_5_ID':series['racer_5_ID'],
                            'racer_6_ID':series['racer_6_ID'],
                            'racer_1_bo':series['racer_1_bo'],
                            'racer_1_mo':series['racer_1_mo'],
                            'racer_2_bo':series['racer_2_bo'],
                            'racer_2_mo':series['racer_2_mo'],
                            'racer_3_bo':series['racer_3_bo'],
                            'racer_3_mo':series['racer_3_mo'],
                            'racer_4_bo':series['racer_4_bo'],
                            'racer_4_mo':series['racer_4_mo'],
                            'racer_5_bo':series['racer_5_bo'],
                            'racer_5_mo':series['racer_5_mo'],
                            'racer_6_bo':series['racer_6_bo'],
                            'racer_6_mo':series['racer_6_mo'],
                            'racer_1_age':racer_1_df.iat[0,3],
                            'racer_1_ave_st':racer_1_df.iat[0,5],
                            'racer_1_doub':racer_1_df.iat[0,4],
                            'racer_1_rank':racer_1_df.iat[0,1],
                            'racer_1_male':racer_1_df.iat[0,2],
                          
                            'racer_2_age':racer_2_df.iat[0,3],
                            'racer_2_ave_st':racer_2_df.iat[0,5],
                            'racer_2_doub':racer_2_df.iat[0,4],
                            'racer_2_rank':racer_2_df.iat[0,1],
                            'racer_2_male':racer_2_df.iat[0,2],
                            
                            'racer_3_age':racer_3_df.iat[0,3],
                            'racer_3_ave_st':racer_3_df.iat[0,5],
                            'racer_3_doub':racer_3_df.iat[0,4],
                            'racer_3_rank':racer_3_df.iat[0,1],
                            'racer_3_male':racer_3_df.iat[0,2],
                          
                            'racer_4_age':racer_4_df.iat[0,3],
                            'racer_4_ave_st':racer_4_df.iat[0,5],
                            'racer_4_doub':racer_4_df.iat[0,4],
                            'racer_4_rank':racer_4_df.iat[0,1],
                            'racer_4_male':racer_4_df.iat[0,2],
                          
                           
                            'racer_5_age':racer_5_df.iat[0,3],
                            'racer_5_ave_st':racer_5_df.iat[0,5],
                            'racer_5_doub':racer_5_df.iat[0,4],
                            'racer_5_rank':racer_5_df.iat[0,1],
                            'racer_5_male':racer_5_df.iat[0,2],
                          
                            'racer_6_age':racer_6_df.iat[0,3],
                            'racer_6_ave_st':racer_6_df.iat[0,5],
                            'racer_6_doub':racer_6_df.iat[0,4],
                            'racer_6_rank':racer_6_df.iat[0,1],
                            'racer_6_male':racer_6_df.iat[0,2] }, index=[''])
    #//////////////////////////////データフレームにadd_dfを追加していく。
    pred_race_df=pred_race_df.append(add_df)
pred_race_df.to_csv('/content/drive/My Drive/pred_tool/asiya/start_list/{}_starts_asiya.csv'.format(date))

In [7]:
type(racer_6_df)

pandas.core.frame.DataFrame

In [8]:
len(racer_5_df)

1

In [9]:
para_df.iloc[-1]

racer_ID             4867.000
racer_rank              2.000
racer_male              1.000
racer_age              29.000
racer_doub_win          0.119
racer_ave_st_time       0.190
Name: 1630, dtype: float64

In [10]:
para_df[(para_df['racer_doub_win']==0.00) & (para_df['racer_ave_st_time']==0.00)].iloc[0]

racer_ID             3051.0
racer_rank              1.0
racer_male              1.0
racer_age              58.0
racer_doub_win          0.0
racer_ave_st_time       0.0
Name: 20, dtype: float64

In [11]:
pred_race_df

,number_race,racer_1_ID,racer_2_ID,racer_3_ID,racer_4_ID,racer_5_ID,racer_6_ID,racer_1_bo,racer_1_mo,racer_2_bo,racer_2_mo,racer_3_bo,racer_3_mo,racer_4_bo,racer_4_mo,racer_5_bo,racer_5_mo,racer_6_bo,racer_6_mo,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_1_male,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_2_male,racer_3_age,racer_3_ave_st,racer_3_doub,racer_3_rank,racer_3_male,racer_4_age,racer_4_ave_st,racer_4_doub,racer_4_rank,racer_4_male,racer_5_age,racer_5_ave_st,racer_5_doub,racer_5_rank,racer_5_male,racer_6_age,racer_6_ave_st,racer_6_doub,racer_6_rank,racer_6_male
,1.0,4477.0,4355.0,4674.0,5029.0,3742.0,5071.0,62.0,14.0,69.0,27.0,75.0,58.0,46.0,40.0,67.0,16.0,59.0,51.0,33.0,0.14,0.588,4,1,37.0,0.21,0.123,2,1,33.0,0.16,0.266,2,1,23.0,0.17,0.230,2,1,46.0,0.17,0.319,2,1,26.0,0.15,0.048,2,1
,2.0,4193.0,4892.0,4588.0,4831.0,4395.0,5107.0,35.0,65.0,29.0,2.0,60.0,19.0,12.0,29.0,38.0,4.0,48.0,20.0,39.0,0.13,0.482,4,1,30.0,0.17,0.287,2,1,29.0,0.12,0.361,2,1,25.0,0.15,0.504,4,1,34.0,0.16,0.320,2,1,21.0,0.19,0.000,1,1
,3.0,4474.0,4329.0,4148.0,4555.0,4932.0,4100.0,65.0,56.0,11.0,1.0,20.0,49.0,33.0,23.0,54.0,9.0,55.0,11.0,33.0,0.17,0.350,3,1,37.0,0.17,0.323,2,1,38.0,0.13,0.514,4,1,31.0,0.19,0.141,2,1,24.0,0.15,0.503,4,1,41.0,0.18,0.376,2,1
,4.0,4522.0,3566.0,4368.0,3747.0,5081.0,4632.0,72.0,43.0,24.0,17.0,58.0,13.0,61.0,60.0,68.0,42.0,56.0,39.0,33.0,0.14,0.586,4,1,50.0,0.20,0.310,2,1,35.0,0.20,0.225,2,1,45.0,0.15,0.381,2,1,22.0,0.17,0.015,2,1,31.0,0.17,0.267,2,1
,5.0,4545.0,4697.0,3297.0,5071.0,3662.0,4463.0,52.0,25.0,50.0,45.0,49.0,53.0,59.0,51.0,25.0,6.0,66.0,38.0,32.0,0.15,0.521,4,1,34.0,0.20,0.292,2,1,53.0,0.18,0.436,3,1,26.0,0.15,0.048,2,1,51.0,0.14,0.408,3,1,34.0,0.16,0.354,2,1
,6.0,3444.0,4736.0,4232.0,3742.0,5107.0,4189.0,23.0,30.0,18.0,18.0,53.0,55.0,67.0,16.0,48.0,20.0,47.0,63.0,51.0,0.16,0.201,2,1,29.0,0.15,0.478,4,1,37.0,0.18,0.210,2,1,46.0,0.17,0.319,2,1,21.0,0.19,0.000,1,1,39.0,0.15,0.474,4,1
,7.0,3434.0,4588.0,4709.0,5041.0,4555.0,5022.0,63.0,41.0,60.0,19.0,16.0,24.0,21.0,36.0,33.0,23.0,17.0,35.0,51.0,0.17,0.350,2,1,29.0,0.12,0.361,2,1,31.0,0.15,0.360,3,1,22.0,0.19,0.142,2,1,31.0,0.19,0.141,2,1,26.0,0.15,0.132,2,1
,8.0,4402.0,4727.0,4346.0,3783.0,4474.0,4180.0,40.0,61.0,30.0,5.0,57.0,64.0,71.0,54.0,65.0,56.0,36.0,12.0,35.0,0.15,0.396,3,1,33.0,0.15,0.201,2,1,35.0,0.17,0.241,2,1,44.0,0.13,0.527,4,1,33.0,0.17,0.350,3,1,37.0,0.16,0.212,2,1
,9.0,4395.0,4477.0,4506.0,5081.0,5073.0,3297.0,38.0,4.0,62.0,14.0,37.0,26.0,68.0,42.0,31.0,21.0,49.0,53.0,34.0,0.16,0.320,2,1,33.0,0.14,0.588,4,1,32.0,0.15,0.430,3,1,22.0,0.17,0.015,2,1,25.0,0.19,0.047,2,1,53.0,0.18,0.436,3,1
,10.0,4424.0,3747.0,4892.0,4189.0,4100.0,5041.0,28.0,62.0,61.0,60.0,29.0,2.0,47.0,63.0,55.0,11.0,21.0,36.0,35.0,0.15,0.470,3,1,45.0,0.15,0.381,2,1,30.0,0.17,0.287,2,1,39.0,0.15,0.474,4,1,41.0,0.18,0.376,2,1,22.0,0.19,0.142,2,1


### 前処理、クラスタリング、ダミー変数化を行ってモデルに突っ込むだけの状態にする

In [12]:
def make_PCA_df(PCA_arr):#PCAで削減したものは二次元配列で帰ってくるので、それをデータフレームにして返す関数
  X=[0]*len(PCA_arr)
  Y=[0]*len(PCA_arr)
  index=0
  for arr in PCA_arr:
    X[index]=arr[0]
    Y[index]=arr[1]
    index+=1
  return pd.DataFrame({'X':X,'Y':Y})

def making_pred_df(df):
  pred_race_df=df
  #pred_race_df=pred_race_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
  result_df=pred_race_df
  result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
  result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
  result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
  result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
  result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
  result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
  result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
  result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
  result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
  result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
  result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
  result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
  result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
  #ダミー変数化
  result_df_dummie=result_df
  race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
  for column, val in race_dummie_df.iteritems():
    result_df_dummie['race_{}'.format(int(column))]=val
  result_df_dummie=result_df_dummie.drop('number_race',axis=1)

  cols=list(result_df_dummie.columns)
  male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

  #===========================新規、性別の取り出し機能が良くなかったため作り直す
  empty_arr=[0]*len(result_df_dummie)
  for col in male_cols:
    for number in np.arange(0,2,1):
      result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
    for column, val in male_dummie_df.iteritems():
      result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
  
  cols=list(result_df_dummie.columns)
  moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
  boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
  #boat もmoterも番号は1~99とする
  numbers=np.arange(1, 100, 1)
  empty_arr=[0]*len(result_df_dummie)
  for col in moter_cols:
    for number in numbers:
      result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
    for column, val in moter_dummie_df.iteritems():
      result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

  #boat番号をダミー化
  for col in boat_cols:
    for number in numbers:
      result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
    for column, val in boat_dummie_df.iteritems():
      result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
  result_df=result_df_dummie



  #クラスタリング
  #分けてみるクラスタの数は[8,10]の2個
  #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
  target_num_cluster=[8,10]
  #test_clustaring_df=train_has_PCA_df
  clustar_target_df=result_df
  clustaring_df=clustar_target_df
  for num_cluster in target_num_cluster:
    pred = KMeans(n_clusters=num_cluster).fit_predict(clustar_target_df)
    clustaring_df['num={}'.format(num_cluster)]=pred

  model_df=clustaring_df

  return model_df

model_df=making_pred_df(pred_race_df)

### 保存済みモデル読み込み

In [13]:
#/content/drive/My Drive/boatrace_BOT_making/asiya_model/asiya_traindata/train_asiya.csv
model4_109 = pickle.load(open('/content/drive/My Drive/boatrace_BOT_making_v2/pred_tool/pickle_data/finalized_model_4_109.sav', 'rb'))
model7_138 = pickle.load(open('/content/drive/My Drive/boatrace_BOT_making_v2/pred_tool/pickle_data/finalized_model_7_138.sav', 'rb'))
model7_145 = pickle.load(open('/content/drive/My Drive/boatrace_BOT_making_v2/pred_tool/pickle_data/finalized_model_7_145.sav', 'rb'))

予測



In [14]:
model4_109_pred=model4_109.predict(model_df)
model7_138_pred=model7_138.predict(model_df)
model7_145_pred=model7_145.predict(model_df)
num_race=np.arange(1,13,1)
#予測結果を出力
pred_df=pd.DataFrame({'num_race':num_race,
                      'pred_4_109':model4_109_pred,
                      'pred_7_138':model7_138_pred})
                     # 'pred_7_145':model7_145_pred})
pred_df.to_csv('/content/drive/My Drive/pred_tool/asiya/pred_data/top2/{}_pred2.csv'.format(date))


top3予測

In [15]:
##/content/drive/My Drive/boatrace_BOT_making/asiya_model/asiya_traindata/train_asiya.csv
model3 = pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com3_dep6_per121.sav', 'rb'))
model4 = pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com4_dep7_per131.sav', 'rb'))
model5 = pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com5_dep8_per122.sav', 'rb'))
model7 = pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com7_dep7_per146.sav', 'rb'))
model13 = pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com13_dep6_per115.sav', 'rb'))
model14= pickle.load(open('/content/drive/My Drive/pred_tool/asiya/pickle_data/model_com14_dep4_per123.sav', 'rb'))
model3_pred=model3.predict(model_df)
model4_pred=model4.predict(model_df)
model5_pred=model5.predict(model_df)
model7_pred=model7.predict(model_df)
model13_pred=model13.predict(model_df)
model14_pred=model14.predict(model_df)

pred_3_df=pd.DataFrame({'num_race':num_race,
                        'pred_3':model3_pred,
                        'pred_4':model4_pred,
                        'pred_5':model5_pred,
                        'pred_7':model7_pred,
                        'pred_13':model13_pred,
                        'pred_14':model14_pred
                        })

#pred_3_df.to_csv('/content/drive/My Drive/boatrace_BOT_making_v2/pred_tool/pred_data/top3/{}_pred3.csv'.format(date))


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


わかりやすい形に直してあげる

In [16]:
def trans_com(target_com):
  #racers_arr=[1,2,3,4,5,6]
  racers_arr=['1','2','3','4','5','6']
  result_com=target_com
  #result_com+=(racer_1 - 1)*20
  racer_1=int(result_com/20)+1
  #next_num=result_com%20
  next_num=result_com-(int(result_com/20)*20)
  racers_arr.remove(str(racer_1))

  racer_2=int(racers_arr[int(next_num/4)])
  next_num=next_num-int(next_num/4)*4
  if next_num==0:
    racer_2=racer_2-1
  racers_arr.remove(str(racer_2))

  racer_3=racers_arr[next_num-1]
  out_text='|{}-{}-{}|'.format(racer_1,racer_2,racer_3)

  return out_text

In [17]:
trans_com(14)

'|1-5-3|'

In [18]:
buy_arr=['']*len(num_race)
target_coms=[3,4,5,7,13,14]
pred_trans_base_df=pd.DataFrame({'pred_3':model3_pred,
                        'pred_4':model4_pred,
                        'pred_5':model5_pred,
                        'pred_7':model7_pred,
                        'pred_13':model13_pred,
                        'pred_14':model14_pred
                        })

i=0
for _, row in pred_trans_base_df.iterrows():
  #preds=list()
  preds=''
  for_i=0
  for val in row.values:
    #print(val)
    
    if val==1:
      text= trans_com(target_coms[for_i])
      #preds.append(text)
      preds+=text
      
    else:
      pass
    for_i+=1
  #buy_arr[i]='/n'.join(preds)
  buy_arr[i]=preds
  i+=1
  print('=====================================')
buy_3_df=pd.DataFrame({'num_race':num_race,
                        'buy_com':buy_arr
                        })
buy_3_df.to_csv('/content/drive/My Drive/pred_tool/asiya/pred_data/top3/{}_pred3.csv'.format(date))

In [19]:
print(i)

12


In [20]:
buy_arr[0]

'|1-5-2||1-5-3|'

In [21]:
pred_3_df

,num_race,pred_3,pred_4,pred_5,pred_7,pred_13,pred_14
0,1,0,0,0,0,1,1
1,2,0,0,0,0,0,0
2,3,0,0,0,0,0,0
3,4,0,0,0,0,0,0
4,5,1,0,0,0,1,1
5,6,0,1,0,0,0,0
6,7,0,0,1,0,0,0
7,8,0,0,0,0,0,0
8,9,0,1,0,0,0,0
9,10,0,0,0,0,0,0


In [22]:
buy_3_df


,num_race,buy_com
0,1,|1-5-2||1-5-3|
1,2,
2,3,
3,4,
4,5,|1-2-5||1-5-2||1-5-3|
5,6,|1-2-6|
6,7,|1-3-2|
7,8,
8,9,|1-2-6|
9,10,


In [23]:
print(trans_com(15))

|1-5-4|
